In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import MarkdownTextSplitter
from langchain_core.documents.base import Document
from langchain.document_loaders import DirectoryLoader

# from langchain_community.document_loaders import TextLoader

In [31]:
# https://www.si.umich.edu/programs/master-applied-data-science/curriculum/mads-courses

courses = {
    "505": "Data Manipulation",
    "502": "Math Methods I",
    "515": "Efficient Data Processing",
    "521": "Visual Exploration of Data",
    "532": "Data Mining I",
    "501": "Being a Data Scientist",
    "511": "SQL and Databases",
    "522": "Information Visualization I",
    "503": "Data Science Ethics",
    "523": "Communicating Data Science Results",
    "542": "Supervised Learning",
    "543": "Unsupervised Learning",
    "611": "Database Architecture & Technology",
    "516": "Big Data: Scalable Data Processing",
    "622": "Information Visualization II",
    "593": "Milestone I",
    "631": "Experiment Design and Analysis",
    "652": "Network Analysis",
    "632": "Data Mining II",
    "673": "Cloud Computing",
    "642": "Deep Learning I",
    "643": "Machine Learning Pipelines",
    "655": "Applied Natural Language Processing",
    "524": "Presenting Uncertainty",
    "601": "Qualitative Inquiry for Data Scientists",
    "630": "Causal Inference",
    "696": "Milestone II",
    "682": "Social Media Analytics",
    "688": "Data Science for Social Good",
    "699": "Capstone",
    # "571": "Business SQL",
    # "602": "Math Methods II",
    # "644": "Reinforcement Learning Algorithms",
    # "680": "Learning Analytics and Educational Data Science",
    # "681": "Health Analytics",
    # "685": "Search and Recommender Systems",
    # "687": "Introduction to Sports Analytics",
}

In [32]:
descriptions = {
    "505": "This course will introduce data manipulation and cleaning techniques using the popular python pandas data science library and introduce the abstraction of the Series and Data Frame as the central data structures for data analysis, along with understanding of how to use functions such as groupby, merge, and pivot tables effectively. By the end of this course, students will be able to take tabular data, clean it, manipulate it, and run basic inferential statistical analyses.",
    "502": "This course covers foundational topics in linear algebra and probability. Important mathematical concepts are introduced or derived in a rigorous manner. Through guided exercises, students will also learn to program with NumPy, the data science library that will be used extensively in subsequent technical courses. ",
    "515": "This course will introduce students to the basics of the linux command-line interface, debugging concepts, basic algorithmic principles such as memoization, recursion, caching, and generators, as well as efficiency and code profiling.",
    "521": "Visual Exploration of Data teaches students how to look for (visually) aggregate patterns within data using the matplotlib library. Students will learn the challenges in visually exploring and representing analytic data with a focus on understanding how statistical measures can be applied.",
    "532": "Data Mining I introduces basic concepts and tasks of data mining. This course focuses on how to formally represent real world information as basic data types (itemsets, matrices and sequences) that facilitate downstream analytics tasks. Students will learn how to characterize each type of data through pattern extraction and similarity measures.",
    "501": "This course explores what expertise, perspectives and ethical commitments applied data scientists bring to projects. We will apply that lens to four phases of data science projects: problem formulation, data acquisition, modeling and analysis, and presentation of results. Through this process students will formulate a statement of who they want to be as data scientists.",
    "511": "This course will introduce the students to beginning and intermediate database concepts. The students will learn the structured query language, the design of data models, loading and normalizing data, how to query databases, and how to measure the performance of various ways of indexing and querying data.",
    "522": "Information Visualization I will initiate information visualization — visual representations of data through interactive systems. Specific focus is on the role of visualization in understanding data, multidimensional data, and how perception, cognition, and good design enhance visualization. This course introduces APIs for visualization construction.",
    "503": "The course introduces the ethical challenges that data scientists face and can help to resolve using case-based reasoning within four domains that are central to data science: data privacy, bias, data provenance, and accountability.",
    "523": "Data scientists need to be able to present their analyses to clients and stakeholders, sometimes \"translating\" to those without statistical or data fluency. You will learn strategies for effective visual, written, and oral communication and create a technical report and oral presentation for your professional portfolio.",
    "542": "Students will learn how to correctly apply, interpret results, and iteratively refine and tune supervised machine learning models to solve a diverse set of problems on real-world datasets. Application is emphasized over theoretical content.",
    "543": "Students will learn how to correctly apply, interpret results, and iteratively refine and tune unsupervised machine learning models to solve a diverse set of problems on real-world datasets. Application is emphasized over theoretical content.",
    "611": "This course will cover advanced techniques in representing and indexing data in JSON and full-text fields. We will also review and cover the performance of database operations across all types of SQL queries. We also compare and contrast relational and non-relational approaches to database and discuss when to use various different database technologies.",
    "516": "This course will introduce students to the use of the Spark data analysis framework for the analysis of Big Data. We will cover the theory and application of MapReduce strategies, the use of Resilient Distributed Datasets in cluster computing, and higher-level abstractions such as DataFrames and SparkSQL, which facilitate efficient analysis of large amounts of data.",
    "622": "This course extends our understanding of information visualization. Leveraging the topics covered in Information Visualization I, we introduce interactive techniques that can be used broadly for visualization. The course will also introduce techniques for visualizing specific data types: temporal, network, hierarchical, and text.",
    "593": "Students will engage in a portfolio project which covers data analysis, manipulation, and visualization, as well as a comprehensive exam to date on learning to date.",
    "631": "Experiment Design and Analysis presents experiment design for laboratory and field experiments. Students will discuss the logic of experimentation and the ways in which experimentation has been — and could be — used to investigate social and technological phenomena. Students will learn how to design experiments and analyze experimental data. ",
    "652": "This course will introduce students to basic network analysis techniques, with an emphasis on developing programming skills to manipulate an analyze real network data using Python. The course includes topics such as network evolution, link prediction, network centrality, models of information diffusion on networks, and community structure.",
    "632": "This course extends Data Mining I and introduces additional data representations and tasks involved in mining real world data, with a particular focus on sequence modeling, time series analysis, and mining data streams. It introduces how to extract patterns, compute similarities/distances of data, and conduct other relevant data mining tasks under these data representations.",
    "673": "This course will serve as an introduction to cloud computing and teach students about cloud infrastructure, cloud networks, management, methods to compare and contrast computing services, and performance, scalability, and availability of cloud resources for data related tasks. Transferring of large datasets around within the cloud to create cloud based pipelines will also be covered. At the end of this course, students should be able to set up cloud based workflows for doing common data science tasks.",
    "642": "This course will introduce basics of neural networks and deep learning. It will focus on developing an understanding of what kind of real-world data problems are amenable to deep learning models. Students will develop familiarity with various commonly used deep learning methods such as Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), and Generative Adversarial Networks (GANs) via hands-on coding exercises and multichoice quizzes.",
    "643": "Students will gain an understanding of how machine learning pipelines function and common issues that occur during the construction and deployment phases. Students will learn how to prototype, test, evaluate, and validate pipelines. By the end, students should be able to build an end-to-end pipeline for supervised machine learning tasks.",
    "655": "This course introduces students to working with language data and how to turn unstructured text into structured information for use with other data science areas. Students will learn how to use common techniques and software libraries for analyzing the semantics and syntax of language.",
    "524": "This course will introduce strategies and techniques for effective uncertainty communication, with a particular focus on uncertainty visualization. A coherent framework for understanding the communication of uncertainty from statistical models will be introduced, as well as strategies for communicating, uncertainty about model specifications themselves.",
    "601": "Qualitative research methodologies are useful to data scientists as a way to gain insight into data sets, to generate hypotheses about data, and to synthesize analysis outcomes. The course will overview basic qualitative research skills including interview protocols, semi-structured interviews, qualitative analysis through affinity diagrams, and synthesis for written reports.",
    "630": "This course introduces common methods of causal inference. Students will learn how to think about appropriate research designs to estimate causal effects. Topics include randomized experiments, regression discontinuity, instrumental variables, and differences-in-differences. We will go through examples using python. Familiarity with basic concepts in probability and statistics is desirable.",
    "696": "Milestone II represents a key assessment point for assessing fundamental data analytics skills and practices applied to realistic data science problems. Students will complete project-based work with guidance from the instructor, as well as a comprehensive exam.",
    "682": "This course will introduce concepts and approaches to mining social media data. It focuses on obtaining and exploring those data, mining networks, and mining texts from social platforms. Students will learn how to apply previously learned data mining concepts in the context of social media data.",
    "688": "This course analyzes the motivations and incentives for people to contribute to public goods. Students will learn how to apply causal inference techniques and social science theories to nudge pro-social behavior and evaluate the impact of such interventions. Application domains include energy conservation, safe driving, contributions to open content (Wikipedia), and open source software.",
    "699": "This is a project-based course in which students propose and build end-to-end data science projects in their domains of interest. Students are asked to demonstrate mastery of data science concepts and methods from their MADS training and produce a creative, original, and technically rigorous portfolio piece. Projects will be supervised by instructors with regular peer review.",
    # "571": "This class will teach you how to handle complex SQL queries from business logic. Students will be presented with a variety of business needs and a database, students will have to respond to the business need using the database to answer the business question. This will cover using pandas to query a database as well."
    # "602": "As a natural continuation of Math Methods I, this course covers more advanced topics in linear algebra and statistics in addition to some introductory optimization. Beyond rigorous mathematical definitions and derivations, students will also learn and explore important applications built upon selected concepts. The assessments in this course will explicitly encourage students to write efficient code that re-implements the mathematical ideas taught in the lectures.",
    # "644": "This course covers the basic principles of reinforcement learning and popular modern reinforcement learning algorithms. Students will develop familiarity with both model-based and model-free reinforcement learning algorithms, including Q-learning, Actor-Critic algorithms, and multi-armed bandit algorithms."
    # "680": "The course examines the application of data science to better understand and improve learning. Anchored in the fields of learning analytics and educational data mining, this course analyzes the unique opportunities and challenges associated with applying data science methods to data stemming from schools, universities, and myriad learning opportunities."
    # "681": "This course analyzes successful deployments and case studies of health analytics. Students will learn how to recognize the role of data analytics in healthcare settings and will be given a broad healthcare challenge to identify the appropriate technical tools to bear on the task. Students will gain experience with consumer informatics / mobile health, electronic health records, medical imaging and public health datasets. They will also gain domain expertise in the areas of stress and physical activity, pharmaceuticals, lung disease and epidemiology.",
    # "685": "This course introduces concepts, methods, and applications of information retrieval, with a focus on search and recommendations. Students will learn algorithms, evaluation metrics, and tools for indexing, ranking, and filtering text data and understand the context of popular industrial applications, including Web search, ads ranking, feeds ranking, and recommender systems.",
    # "687": "In this course students will study how supervised machine learning techniques are applied in the domain of sports analytics including in individual sporting events, team events, and emerging wearable sensor technologies. Students will engage in applying their knowledge of machine learning through a hands-on competition in one or more sports analytics domains.",
}

In [44]:
from datetime import datetime
import re
from typing import List

from langchain_core.documents import Document
from langchain_community.document_loaders.base import BaseLoader


class SyllabusLoader(BaseLoader):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self) -> List[Document]:
        try:
            with open(self.file_path, encoding="UTF-8") as f:
                text = f.read()
        except Exception as e:
            raise RuntimeError(f"Error loading {self.file_path}") from e

        heading_pattern = r"^(#{1,6})\s*(.*)$"
        headings = [
            (match.start(), match.group())
            for match in re.finditer(heading_pattern, text, flags=re.MULTILINE)
        ]

        sections = []
        for i in range(len(headings)):
            start, heading = headings[i]
            end = headings[i + 1][0] if i + 1 < len(headings) else len(text)
            section_text = text[start:end].replace(heading, "").strip()

            file = self.file_path.split("/")[-1]
            date, course = file.split("_")
            course_number = course.replace(".md", "")

            metadata = {
                "source": file,
                "heading": heading.replace("#", "").strip(),
                "course_number": f"SIADS {course_number}",
                "course_title": courses[course_number],
                "course_date": datetime.strptime(date, "%Y-%m").strftime("%B %Y"),
            }

            sections.append([Document(page_content=section_text, metadata=metadata)])

        return sections

In [45]:
loader = DirectoryLoader("./parsed_syllabi", glob="*.md", loader_cls=SyllabusLoader)

doc = loader.load()

len(doc)

571

In [47]:
doc[40]

[Document(page_content='Refer to the [Screen reader configuration for Jupyter Notebook Content](https://docs.google.com/document/d/1ct4BShNTYVU2J_oYeuTTsODSAFlEhtODXMlfc4-t5PM/edit#heading=h.t003rxazhbx3) document to learn accessibility tips for Jupyter Notebooks.', metadata={'source': '2023-11_682.md', 'heading': 'Accessibility', 'course_number': 'SIADS 682', 'course_title': 'Social Media Analytics', 'course_date': 'November 2023'})]

In [7]:
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(doc)

len(texts)

485

In [8]:
texts[32]

Document(page_content='Online access to these textbooks are provided through the University of Michigan Library. You may be asked to sign in with your UMich uniquename and password to access these materials.\n\n### **Technology Requirements (unique to this course)**\n\n_None_\n\n### **Accessibility**\n\n[Screen reader configuration for Jupyter Notebook Content](https://docs.google.com/document/d/1ct4BShNTYVU2J_oYeuTTsODSAFlEhtODXMlfc4-t5PM/edit?usp=sharing)\n\n### **Learning Outcomes**', metadata={'source': 'parsed_syllabi/2021-12_532.md'})

In [10]:
persist_directory = "chromadb_test"

In [9]:
from huggingface_hub import snapshot_download

# Running hugging face hub in a cell and downloading cache
snapshot_download(repo_id="sentence-transformers/all-MiniLM-L6-v2")

/Users/Pat/Documents/umich/2024-02 SIADS 699/capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 23.4MB/s]s]
Fetching 16 files: 100%|██████████| 16/16 [00:00<00:00, 57.33it/s]


'/Users/Pat/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/44eb4044493a3c34bc6d7faae1a71ec76665ebc6'

In [11]:
vectordb = Chroma.from_documents(
    documents=texts,
    collection_name="MADS",
    # Chose the embedding you want to use
    embedding=embeddings_open,
    persist_directory=persist_directory,
)

In [12]:
# Save to disk
vectordb.persist()
vectordb = None

In [13]:
# Reload from disk
vectordb = Chroma(
    "MADS",
    persist_directory=persist_directory,
    embedding_function=embeddings_open,
    collection_metadata={"hnsw:space": "cosine"},
)

In [14]:
# Create retriever
retriever = vectordb.as_retriever(
    # search_type="mmr",
    # search_kwargs={'k': 5, 'fetch_k': 20}
    # search_type="similarity_score_threshold",
    # search_kwargs={'score_threshold': 0.8}
)

In [15]:
docs = retriever.get_relevant_documents("Who is the course manager for SIADS 699?")
len(docs)

4

In [16]:
docs

[Document(page_content='1. American Civil Liberties Union. (2004). "Scary Pizza." (video.) 01:42. Available online: <https://www.youtube.com/watch?v=33CIVjvYyEk>\n2. Raji, Deborah. (December 10, 2020). " [How our data encodes systematic racism](https://www.technologyreview.com/2020/12/10/1013617/racism-data-science-artificial-intelligence-ai-opinion/) ." MIT Technology Review.\n3. Wallach, Hanna. (December 14, 2014). [Big Data, Machine Learning, and the Social Sciences: Fairness, Accountability, and Transparency.](https://medium.com/@hannawallach/big-data-machine-learning-and-the-social-sciences-927a8e20460d) Medium.com.', metadata={'source': 'parsed_syllabi/2022-03_503.md'}),
 Document(page_content='- Correctly apply and interpret results from clustering methods in scikit-learn, including k-means, agglomerative clustering, hierarchical clustering, and DBSCAN.\n- Understand the use of topic modeling (Latent Dirichlet Allocation and Non-Negative Matrix Factorization forms) and best prac

In [17]:
def process_llm_response(llm_response):
    # print(llm_response['result'])
    print("\n\nSources:")
    for source in llm_response["source_documents"]:
        print(source.metadata["source"])

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_open,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

In [24]:
# Question
query = "Tell me what I need to turn in for the capstone project"
# query = "Which course can I take to learn about degree centrality?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...


ValueError: Error raised by inference API HTTP code: 404, {"error":"model 'Llama2' not found, try pulling it first"}